In [51]:
import torch
import torch.nn as nn
import scipy.stats as stats
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import optim
import matplotlib.pyplot as plt

In [52]:
from icnnet import ICNNet
from mydataset import MyDataset, get_gaussian_dataset, get_gaussian_transport_dataset
from toy_data_dataloader_gaussian import generate_gaussian_dataset, get_dataset, generate_dataset, generate_traj
from train_picnn import PICNNtrain
from train_wasserstein import train_wasserstein
from train_makkuva import train_makkuva, train_makkuva_epoch
from visualization import plot_transport
from gaussian_transport import get_gaussian_transport

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## __Generate dataset__


In [59]:
#dataset = get_dataset(d=2, r=100, N=500) #valou
#dataset = generate_gaussian_dataset(d=2, r=400, N=10000) #thomas
#dataset = generate_traj(d = 2, r = 100, N=50)
dataset = generate_dataset(d=2, r=500, N=50)
gaussian_dataset = get_gaussian_dataset(dataset)
gaussian_transport_dataset = get_gaussian_transport_dataset(gaussian_dataset)

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0158,  0.0075],
         [ 0.0124,  0.0023],
         [ 0.0056,  0.0156],
         ...,
         [ 0.0218,  0.0102],
         [ 0.0068,  0.0029],
         [ 0.0469,  0.0016]],

        [[ 0.0513,  0.0102],
         [ 0.0186,  0.0070],
         [ 0.0148,  0.0169],
         ...,
         [ 0.0231,  0.0105],
         [ 0.0204,  0.0102],
         [ 0.0637,  0.0131]],

        ...,

        [[ 0.1073,  0.0142],
         [-0.0540,  0.0111],
         [ 0.0831, -0.0050],
         ...,
         [-0.0478,  0.0820],
         [ 0.0802,  0.0857],
         [ 0.2037,  0.0414]],

        [[ 0.0898, -0.0050],
         [-0.0602,  0.0042],
         [ 0.0680, -0.0291],
         ...,
         [-0.0488,  0.0803],
         [ 0.0736,  0.0634],
         [ 0.2004,  0.0385]],

        [[ 0.0937, -0.0014],
       

ValueError: array must not contain infs or NaNs

In [ ]:
def get_mean(batch):
    means = torch.mean(batch, dim=1)
    average_mean = torch.mean(means, dim=0)
    return(average_mean)

def get_covariance(batch):
    n = batch.size(1) - 1
    mean = torch.mean(batch, dim=1, keepdim=True)
    batch = batch - mean  # Centering the data
    cov = torch.matmul(batch.transpose(1, 2), batch) / n
    return(torch.mean(cov, dim=0))

mean1 = get_mean(dataset.X)
cov1 = get_covariance(dataset.X)
mean2 = get_mean(dataset.Y)
cov2 = get_covariance(dataset.Y)

def init_z_f(x):
    #return(x)
    #return x
    #return (1/2) * torch.norm(x, dim=-1, keepdim=True)**2
    return(get_gaussian_transport(u=x, cov1 = cov1, cov2 = cov2, m1=mean1, m2=mean2))

def init_z_g(x) :
    #return(x)
    #return x
    #return (1/2) * torch.norm(x, dim=-1, keepdim=True)**2
    return(get_gaussian_transport(u=x, cov1 = cov2, cov2 = cov1, m1=mean2, m2=mean1))

## __Initialization__

### __PICNN training__

In [ ]:
input_size = 2
layer_sizes = [input_size,64, 64, 64, 1]
n_layers = len(layer_sizes)

In [ ]:
import torch.nn.functional as F

# def get_embedding(C, c):
#     scalar_product = torch.matmul(c.float(), C.t().float())
#     embedding = F.softmax(scalar_product, dim=1)
#     return(embedding)

context_layer_sizes = [12] + [64] * (n_layers-1)

In [ ]:
model_init_f = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')
model_init_g = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')

In [9]:
n_epoch = 200
lr = 0.001

print('training f')
gaussian_transport_dataloader = DataLoader(gaussian_transport_dataset, batch_size=250, shuffle=True)
PICNNtrain(model_init_f, gaussian_transport_dataloader, init_z_f, lr=lr, epochs=n_epoch)
#PICNNtrain(model_init_f, gaussian_transport_dataloader, init_z_f, lr=lr, epochs=n_epoch)
#PICNNtrain(model_init_f, gaussian_transport_dataloader, lr=0.0001, epochs=1, init_z = lambda x: x)

print('training g')
reversed_gaussian_dataset = MyDataset(gaussian_dataset.Y, gaussian_dataset.C, gaussian_dataset.X)
gaussian_transport_dataset_reversed = get_gaussian_transport_dataset(reversed_gaussian_dataset)
gaussian_transport_dataloader_reversed = DataLoader(gaussian_transport_dataset_reversed, batch_size=250, shuffle=True)
#PICNNtrain(model_init_g, gaussian_transport_dataloader_reversed, lr=0.0001, epochs=25, init_z = lambda x: (1/2) * torch.norm(-x, dim=-1, keepdim=True)**2)
PICNNtrain(model_init_g, gaussian_transport_dataloader_reversed, init_z_g, lr=lr, epochs=int(n_epoch/2))

training f
Epoch 1/200 Loss: 261.4753112792969
Epoch 2/200 Loss: 232.5139617919922
Epoch 3/200 Loss: 200.73245239257812
Epoch 4/200 Loss: 167.34132385253906
Epoch 5/200 Loss: 135.1083526611328
Epoch 6/200 Loss: 109.26573944091797
Epoch 7/200 Loss: 98.717529296875
Epoch 8/200 Loss: 112.18524169921875
Epoch 9/200 Loss: 131.29757690429688
Epoch 10/200 Loss: 132.34042358398438
Epoch 11/200 Loss: 120.81974029541016
Epoch 12/200 Loss: 108.17781829833984
Epoch 13/200 Loss: 100.48158264160156
Epoch 14/200 Loss: 98.4637680053711
Epoch 15/200 Loss: 100.23783874511719
Epoch 16/200 Loss: 103.52745819091797
Epoch 17/200 Loss: 106.62254333496094
Epoch 18/200 Loss: 108.57073974609375
Epoch 19/200 Loss: 109.026123046875
Epoch 20/200 Loss: 108.06221771240234
Epoch 21/200 Loss: 106.02684020996094
Epoch 22/200 Loss: 103.43756866455078
Epoch 23/200 Loss: 100.8939208984375
Epoch 24/200 Loss: 98.97386169433594
Epoch 25/200 Loss: 98.09410095214844
Epoch 26/200 Loss: 98.3560562133789
Epoch 27/200 Loss: 99.454

In [10]:
state_dict_init_f = model_init_f.state_dict()
state_dict_init_g = model_init_g.state_dict()

###### Dorseuil

## __Makkuva__

In [11]:
# state_dict_init_f = torch.load('trained_models/training22/models/model_f_0.pth')
# state_dict_init_g = torch.load('trained_models/training22/models/model_g_0.pth')

In [12]:
ICNNf = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')
ICNNg = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')

# Load the state dictionary into ICNNf and ICNNg
ICNNf.load_state_dict(state_dict_init_f)
ICNNg.load_state_dict(state_dict_init_g)

<All keys matched successfully>

In [13]:
n_points = 2000
test = 14

filepath_pth_f = 'trained_models/training22/models/model_f_'
filepath_pth_g = 'trained_models/training22/models/model_g_'

filepath_plt_f = 'trained_models/training22/plots/model_f_'
filepath_plt_g = 'trained_models/training22/plots/model_g_'

import os
os.makedirs(filepath_pth_f, exist_ok=True)
os.makedirs(filepath_pth_g, exist_ok=True)

os.makedirs(filepath_plt_f, exist_ok=True)
os.makedirs(filepath_plt_g, exist_ok=True)

In [14]:
filename_pth_f = filepath_pth_f + str(0) + '.pth'
filename_pth_g = filepath_pth_g + str(0) + '.pth'
torch.save(ICNNf.state_dict(), filename_pth_f)
torch.save(ICNNg.state_dict(), filename_pth_g)

In [15]:
filename_plt_f = filepath_plt_f + str(0) + '.png'
filename_plt_g = filepath_plt_g + str(0) + '.png'
plot_transport(dataset, test, ICNNf, ICNNg, init_z_f = init_z_f, init_z_g = init_z_g, filename_f = filename_plt_f, filename_g = filename_plt_g, n_points=n_points)

<Figure size 640x480 with 0 Axes>

In [16]:
dataloader = DataLoader(dataset, batch_size=501, shuffle=True)

loss_f = list()
loss_g = list()

prev_param_f = [param.clone().detach() for param in ICNNf.parameters()]
prev_param_g = [param.clone().detach() for param in ICNNg.parameters()]

for epoch in range(1, 101) :
    print('epoch :', epoch)
    mean_loss_f, mean_loss_g, prev_param_f, prev_param_g = train_makkuva_epoch(ICNNf=ICNNf, ICNNg=ICNNg, prev_param_f=prev_param_f, prev_param_g=prev_param_g, dataloader = dataloader, init_z_f = init_z_f, init_z_g = init_z_g, lr=0.0005, train_freq_g=5, train_freq_f=1, regularize_f = True, regularize_g = True, lambda_proximal = 0.0001)
    #mean_loss_f, mean_loss_g = train_makkuva_epoch(ICNNf, ICNNg, None, None, dataloader, init_z_f = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2, init_z_g = lambda x: (1/2) * torch.norm(-x, dim=-1, keepdim=True)**2, lr=0.0001, train_freq_g=10, train_freq_f=1, gaussian_transport=False)

    loss_f.append(mean_loss_f)
    loss_g.append(mean_loss_g)

    filename_pth_f = filepath_pth_f + str(epoch) + '.pth'
    filename_pth_g = filepath_pth_g + str(epoch) + '.pth'
    torch.save(ICNNf.state_dict(), filename_pth_f)
    torch.save(ICNNg.state_dict(), filename_pth_g)

    filename_plt_f = filepath_plt_f + str(epoch) + '.png'
    filename_plt_g = filepath_plt_g + str(epoch) + '.png'
    plot_transport(dataset, test, ICNNf, ICNNg, init_z_f=init_z_f, init_z_g=init_z_g, filename_f = filename_plt_f, filename_g = filename_plt_g, n_points=n_points)  

epoch : 1
4.4833221435546875
2.674133539199829
8.590400695800781
2.7793331146240234
1.1766952276229858
loss_g: 1.1766952276229858, loss_f: 8.093396186828613
epoch : 2
5.18220329284668
1.373329520225525
0.762019157409668
2.355783224105835
1.8270970582962036
loss_g: 1.8270970582962036, loss_f: 10.272769927978516
epoch : 3
1.380231499671936
0.33704113960266113
0.35730239748954773
0.3855452835559845
0.5554842352867126
loss_g: 0.5554842352867126, loss_f: 8.232588768005371
epoch : 4
0.07355661690235138
-0.312826931476593
-0.45284754037857056
-0.5347052812576294
-0.622500479221344
loss_g: -0.622500479221344, loss_f: 3.8914315700531006
epoch : 5
-1.651500940322876
-2.235330581665039
-2.9525442123413086
-3.867929697036743
-5.10163688659668
loss_g: -5.10163688659668, loss_f: 2.6839659214019775
epoch : 6
-3.7869927883148193
-4.540798187255859
-5.491685390472412
-6.722943305969238
-8.361311912536621
loss_g: -8.361311912536621, loss_f: -1.7186726331710815
epoch : 7
40.667232513427734
10.06761646270

<Figure size 640x480 with 0 Axes>

In [17]:
l_test = [12, 13, 14, 15, 16, 17, 18, 19]
for epoch in [0,100]:
    print('epoch :', epoch)

    filename_pth_f = filepath_pth_f + str(epoch) + '.pth'
    filename_pth_g = filepath_pth_g + str(epoch) + '.pth'

    state_dict_init_f = torch.load(filename_pth_f)
    state_dict_init_g = torch.load(filename_pth_f)

    ICNNf.load_state_dict(state_dict_init_f)
    ICNNg.load_state_dict(state_dict_init_g)

    for test in l_test :
        filename_plt_f = filepath_plt_f + 'test_' + str(test) + '_'  +str(epoch) + '.png'
        filename_plt_g = filepath_plt_g + 'test_' + str(test) + '_' + str(epoch) + '.png'
        plot_transport(dataset, test, ICNNf, ICNNg, init_z_f=init_z_f, init_z_g=init_z_g, filename_f = filename_plt_f, filename_g = filename_plt_g, n_points=n_points)  

epoch : 0
epoch : 100


<Figure size 640x480 with 0 Axes>

In [18]:
print(filename_plt_f)

trained_models/training22/plots/model_f_test_19_100.png


In [19]:
for ele in loss_f:
    print(ele)

# print('stop')

# for ele in loss_g:
#     print(ele)

8.093396186828613
10.272769927978516
8.232588768005371
3.8914315700531006
2.6839659214019775
-1.7186726331710815
13.64122486114502
10.566709518432617
6.840768814086914
3.380632162094116
0.5386691689491272
0.2413329929113388
4.414255142211914
15.303526878356934
12.383905410766602
9.8849515914917
7.54427433013916
5.517082691192627
3.803241014480591
2.508186101913452
1.9569975137710571
2.748490333557129
3.599273204803467
-7.30112886428833
-4.516995906829834
3.385284900665283
4.319328784942627
3.0157418251037598
1.8701128959655762
1.9784809350967407
2.3573877811431885
2.3136353492736816
1.7999359369277954
0.9664315581321716
0.1082933098077774
-0.5001576542854309
-0.773777425289154
0.41054677963256836
8.624581336975098
73.83362579345703
11.94672966003418
13.563788414001465
13.283452987670898
11.990884780883789
10.807262420654297
9.792586326599121
8.917327880859375
8.148096084594727
7.391578197479248
6.57034158706665
5.603031158447266
4.442676067352295
3.116647958755493
1.8470609188079834
0.